In [85]:
import time

# Первая задача. Finding a Motif in DNA.

Для поиска подстроки существует огромное разнообразие алгоритмов.Для сравнения я взяла алгоритмы Рабина-Карпа, Кнута-Морриса-Пратта. В среднем они обладают сложностью соответственно: O(H+n), O(2H), где H — длина строки, в которой ведётся поиск, n — длина шаблона поиска.

In [131]:
with open("E_coli.txt", "r") as file:
    s=file.read()
print(len(s))
t="AAATGCAACACGTTTTATCCGTTCTGGACTTCACCCGCTAACCAACGCGC"
print(len(t))

4639675
50


С помощью алгоритма Д. Кнута, Д. Мориса и В. Пратта.

In [138]:
def prefix(s):
    p = [0] * len(s)
    for i in range(1, len(s)):
        k = p[i - 1]
        while k > 0 and s[k] != s[i]:
            k = p[k - 1]
        if s[k] == s[i]:
            k += 1
        p[i] = k
    return p
def kmp(string, substring):
    """
    find all locations of substring in string
    
    Parametrs:
    string: string, in which you find substring
    substring: string, which you find
    Return: string with all locations (space-separated) of substring in string 
    """
    new_str=substring+"#"+string
    index = []
    pi = prefix(new_str)
    for i in range (len(substring)+1, len(new_str)):
        if pi[i]==len(substring):
            index.append(str(i-2*len(substring)+1))
    return " ".join(index)

In [132]:
start = time.time()
kmp(s,t)
end = time.time()
print(end - start)
time_elapsed = end-start
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

2.1270699501037598
Training complete in 0m 2s


С помощью алгоритма Рабина-Карпа. Рассмотрела 2 способа хэширования. По времени работы функции выбрала первый способ, для которого используется класс Hash.

In [133]:
class Hash:
    def __init__(self, string, size):
        self.str  = string
        self.hash = 0
        for i in range(size):
            self.hash += ord(self.str[i])
        self.init = 0
        self.end  = size

    def update(self):
        if self.end <= len(self.str) -1:
            self.hash -= ord(self.str[self.init])
            self.hash += ord(self.str[self.end])
            self.init += 1
            self.end  += 1

    def digest(self):
        return self.hash

    def text(self):
        return self.str[self.init:self.end]
class Hash1:
    def __init__(self, string, size):
        self.str  = string
        self.init = 0
        self.end  = size
        self.hash = 0
        d={"a":0,"c": 1, "g":2, "t":3}
        m=size-1
        for i in range(size):
            self.hash += d[self.str[i]]*4**m
            m=m-1



    def update(self):
        if self.end <= len(self.str) -1:
            self.hash -= d[self.str[self.init]]*4**(size-1)
            self.hash *=4
            self.hash += d[self.str[self.end]]
            self.init += 1
            self.end  += 1

    def digest(self):
        return self.hash

    def text(self):
        return self.str[self.init:self.end]
    
def rabin_karp(string, substring):
    """
    find all locations of substring in string
    
    Parametrs:
    string: string, in which you find substring
    substring: string, which you find
    Return: all locations of substring in string
    """
    string=string.lower()
    substring=substring.lower()
    array=[]
    
    if substring == None or string == None:
        return -1
    if substring == "" or string == "":
        return -1

    if len(substring) > len(string):
        return -1

    hs = Hash(string, len(substring))
    hsub = Hash(substring, len(substring))
    hsub.update()
    for i in range(len(string)-len(substring)+1):
        if hs.digest() == hsub.digest():
            if hs.text() == substring:
                array.append(str(i+1))
        hs.update()
    a=" ".join(array)
    return a
    if array==[]:
        return -1
    

In [134]:
start = time.time()
rabin_karp(s,t)
end = time.time()
print(end - start)
time_elapsed = end-start
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

9.714576721191406
Training complete in 0m 10s


Для всех строк и шаблонов, рассмотренных мною, быстрее работала реализация алгоритма Кнута-Морриса-Пратта.

Наглядный пример работы функции.

In [140]:
kmp('GATATATGCATATACTT','ATAT')

'2 4 10'

# Вторая задача. Genome Assembly with Perfect Coverage.

In [1]:
def circular_string(set_string):
    """Return a cyclic superstring of minimal length containing the reads
    
    Parametrs: 
    set_string: list of strings, which are DNA k-mers (k<50) taken from the same strand of a circular chromosome. All k-mers from this strand of the chromosome are present, and their de Bruijn graph consists of exactly one simple cycle.
    Return the string with the cyclic superstring of minimal length containing the reads (thus corresponding to a candidate cyclic chromosome).
    """
    d={}
    #form de Bruijn graph
    k=int(len(set_string[0]))
    for i in set_string:
        for j in set_string:
            if (i[1:k]==j[0:k-1] and i!=j):
                d[i]=j #in dict key and value forms edge of de Bruijn graph
    #read cyclic superstring
    q=list(d.keys())[0]
    s=q
    for i in range(len(d)-(k)):
        q=d[q]
        s=s+q[-1]
    return s

Пример работы функции.

In [2]:
set_string=['ATTAC','TACAG','GATTA', 'ACAGA','CAGAT','TTACA', 'AGATT']
circular_string(set_string)

'ATTACAG'

# Третья задача. Bellman-Ford Algorithm.

In [73]:
def func_ford_bellman(list_edges):
    """
    Compute single-source shortest distances in a directed simple graph with possibly negative edge weights (but without negative cycles).
    
    Parametrs:
    list_edges : The first line contains two numbers, the number of vertices n and the number of edges m, each of the following m lines contains an edge given by two vertices and its weight. 
    Return:
    An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). If i is not reachable from 1 set D[i] to x
    """
    #form list_edges for our goal
    couples=list_edges.split("\n")
    number_vertex = int(couples[0].split(' ')[0])
    prepared_list_edges=[]
    for i in couples[1:]:
        prepared_list_edges.append(tuple(map(int, i.split(" "))))
        
    d=['x']*number_vertex
    d[0]=0
    for i in range(number_vertex-1):
        for u,v,w in prepared_list_edges:
            if d[u-1]!= 'x':
                if d[v-1] == 'x':
                    d[v-1]=d[u-1]+w
                else:
                    d[v-1]=min(d[v-1], d[u-1]+w)
    return d  

Пример работы функции.

In [74]:
list_edges="9 13\n1 2 10\n3 2 1\n3 4 1\n4 5 3\n5 6 -1\n7 6 -1\n8 7 1\n1 8 8\n7 2 -4\n2 6 2\n6 3 -2\n9 5 -10\n9 4 7"
func_ford_bellman(list_edges)

[0, 5, 5, 6, 9, 7, 9, 8, 'x']

# Четвертая задача. Connected Components.

In [144]:
def help_DFS(start, k, adjacency_list, visited):
    """
    exam one component
    """
    visited[start]=True
    if adjacency_list[start]:
        for v in adjacency_list[start]:
            if not visited[v]:
                help_DFS(v,k, adjacency_list, visited)
def solve_DFS(edge_list):
    """
    Compute the number of connected components in a given undirected graph
    
    Parametrs:
    edge_list: The first line contains two numbers, the number of vertices n and the number of edges m, each of the following m lines contains an edge given by two vertices.
    Returns: the number of connected components in the graph
    """
    #form adjacency_list
    couples=edge_list.split("\n")
    n_vertex, n_edges=couples[0].split(" ")
    adjacency_list=[None]*(int(n_vertex)+1)
    for i in couples[1:]:
        first_vertex=int(i.split(" ")[0])
        second_vertex=int(i.split(" ")[1])
        if not adjacency_list[first_vertex]:
            adjacency_list[first_vertex]=[second_vertex]
        else:
            adjacency_list[first_vertex].append(second_vertex)
        if not adjacency_list[second_vertex]:
            adjacency_list[second_vertex]=[first_vertex]
        else:
            adjacency_list[second_vertex].append(first_vertex)
            
    #compute number of connected components
    k=-1
    ncomp=0
    visited=[False]*(int(n_vertex)+1)
    for i in range(1, int(n_vertex)):
        if not visited[i]:
            k+=1
            ncomp+=1
            help_DFS(i,k, adjacency_list, visited)
    return ncomp

Пример работы функции.

In [147]:
edge_list="12 13\n1 2\n1 5\n5 9\n5 10\n9 10\n3 4\n3 7\n3 8\n4 8\n7 11\n8 11\n11 12\n8 12"
solve_DFS(edge_list)

3

# Пятая задача. Quick Sort.

In [42]:
def QuickSort(array):
    """sort array
    
    Parametrs:
    array: array A[1..n] of integers from −10**5 to  10**5
    Returns: A sorted array A[1..n].
    """
    if len(array) <= 1:
        return array
    else:
        q = array[int(len(array)/2)-1]
        L = [elem for elem in array if elem < q]
        M = [q] * array.count(q)
        R = [elem for elem in array if elem > q] 
        return QuickSort(L) + M + QuickSort(R)

Пример работы функции.

In [43]:
a=[5, -2, 4, 7 ,8, -10, 11]
QuickSort(a)

[-10, -2, 4, 5, 7, 8, 11]